## Introduction

This is the best exercise of all. In this exercise we will choose the **best classification model** with the **best performing parameters** for our Patient_No_Show data.

We will use **GridSearchCV** and **RandomizedGridSearchCV** to select the best parameters.

We will create a **model-parameter-grid** which will provide us the best scoring parameters for every model, helping us choose the best model for **Patient No_Show** prediction

As a *recap*, we have the following **Supervised Learning Classifiers/Models**
1. LogisticRegression
2. DecisionTree / RandomForestClassifier
3. Support Vector Machine (SVC)
4. Naive Bayes
    - GaussianNB
    - MultinomialNB

We will be testing all these models with multiple parameters to identify the best performer.

**Note**: Previously we had identified **DecisionTree/RandomForestClassifier** as the better performing model with an *accuracy score* score of **74.6%**. We will use this as a benchmark
